# Document AI: Fine-tuning Donut for document-parsing using Hugging Face Transformers 

In this blog, you will learn how to fine-tune [Donut-base](https://huggingface.co/naver-clova-ix/donut-base) for document-understaind/document-parsing using Hugging Face Transformers.We are going use all of the great features from the Hugging Face ecosystem like model versioning and experiment tracking.

As dataset we will use the [SROIE](https://github.com/zzzDavid/ICDAR-2019-SROIE) dataset a collection of 1000 scanned receipts including their OCR. More information for the dataset can be found at the [repository](https://github.com/zzzDavid/ICDAR-2019-SROIE).

You will learn how to:

1. [Setup Development Environment](#1-setup-habana-gaudi-instance)
2. [Load and process the dataset](#2-load-and-process-the-dataset)
3. [Create a `Trainer` and define Hyperparameters](#3-create-a-gauditrainer-and-an-run-single-hpu-fine-tuning)
4. [Run training and evaluation](#4-run-distributed-data-parallel-training-with-gauditrainer)

Before we can start make sure you have a [Hugging Face Account](https://huggingface.co/join) to save artifacts and experiments. 

## Quick intro: Document Understanding Transformer (Donut) by ClovaAI

Document Understanding Transformer (Donut) is a new Transformer model for OCR-free document understanding. It doesn't require an OCR engine to process scanned documents, but is achieving state-of-the-art performances on various visual document understanding tasks, such as visual document classification or information extraction (a.k.a. document parsing). 
Donut is a multimodal sequence-to-sequence model with a vision enconder ([Swin Transformer](https://huggingface.co/docs/transformers/v4.21.2/en/model_doc/swin#overview)) and text decoder ([BART](https://huggingface.co/docs/transformers/v4.21.2/en/model_doc/bart)). The encoder receives the images and computes it into a embedding, which is then passed to the decoder, which generates a sequence of tokens.

![donut](../assets//donut.png)

* Paper: https://arxiv.org/abs/2111.15664
* Official repo:  https://github.com/clovaai/donut

--- 

Now we know how Donut works, so lets get started. 🚀

## 1. Environment setup

Our first step is to install the Hugging Face Libraries including transformers and datasets. Running the following cell will install all the required packages.

_Note: As the time of writing this Donut is not yet included in the PyPi version of Transformers, so we need it to install from the `main` branch. Donut will be added in version `4.22.0`._


In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git 
# !pip install -q "transformers>=4.22.0" # comment in when version is released
!pip install -q datasets sentencepiece tensorboard

In [ ]:
# install git-fls for pushing model and logs to the hugging face hub
!sudo apt-get install git-lfs


This example will use the [Hugging Face Hub](https://huggingface.co/models) as a remote model versioning service. To be able to push our model to the Hub, you need to register on the [Hugging Face](https://huggingface.co/join). 
If you already have an account you can skip this step. 
After you have an account, we will use the `notebook_login` util from the `huggingface_hub` package to log into our account and store our token (access key) on the disk. 

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## 2. Load and process the dataset

As dataset we will use the [SROIE](https://github.com/zzzDavid/ICDAR-2019-SROIE) dataset a collection of 1000 scanned receipts including their OCR. The dataset is available on Hugging Face so we can load the dataset using `load_dataset` method. 


In [ ]:
from datasets import load_dataset

datasat = load_dataset("darentang/sroie")